# Task 1

## 그래디언트 클리핑

> Gradient Clipping

그래디언트 폭주(exploding gradient) 문제를 줄이기 위한 방법으로, 역전파 단계에서 그래디언트 값이 특정 임계값(threshold)을 넘지 않도록 잘라낸다.

## 심층 신경망 재사용

규모가 매우 큰 DNN 모델 학습 시 새로 학습하는 작업은 느리다. 이때 기존 학습된 모델이 존재하면 하위 층(lower layer)을 가져와 재사용하는 것이 학습 속도를 빠르게 하고 학습에 필요한 데이터 세트도 적다. 이러한 방법을 Transfer Learning이라고 한다.

이때 재사용에 사용하지 않을 층은 동결(freezing)하고, 사용할 층은 학습 가능한 상태로 유지한다. 동결된 층은 학습 과정에서 변하지 않으므로 가장 마지막에 동결된 층 출력을 캐싱하여 활용하면 학습 속도가 개선된다.

In [4]:
# 기존 모델 생성 및 저장
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# 기존 모델 생성
# 입력 층에서 지난 PR 피드백을 반영하여 Input() 함수를 사용하도록 변경하였습니다.
def create_base_model():
    model = Sequential([
        Input(shape=(784,)),
        Dense(128, activation='relu', name='hidden1'),
        Dense(64, activation='relu', name='hidden2'),
        Dense(32, activation='relu', name='hidden3'),
        Dense(10, activation='softmax', name='output'),
    ])
    return model

# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# 모델 생성 및 컴파일
base_model = create_base_model()
base_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# 모델 학습
base_model.fit(x_train, y_train, batch_size=64, epochs=5)

# 모델 저장
base_model.save('./my_model.keras')
# 'my_model.ckpt'는 Keras에서 'h5' 형식으로 저장됩니다.
# 오류 메시지에 따르면 `.keras` 포맷 사용을 권장하여 변경하였습니다.

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8347 - loss: 0.5456
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9612 - loss: 0.1316
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9747 - loss: 0.0832
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9815 - loss: 0.0627
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9854 - loss: 0.0470


In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# 저장된 기존 모델('my_model.keras') 호출
base_model = load_model("./my_model.keras")

# 새로운 입력 레이어를 정의 (입력 형태: (784,))
input_layer = Input(shape=(784,))

# 기존 모델에서 hidden1부터 hidden3까지를 가져와 연결
hidden1 = base_model.get_layer("hidden1")
hidden2 = base_model.get_layer("hidden2")
hidden3 = base_model.get_layer("hidden3")

# hidden1과 hidden2 레이어 동결
for layer in ["hidden1", "hidden2"]:
  base_model.get_layer(layer).trainable = False

# 새로운 hidden4와 logits 레이어 추가
hidden4 = Dense(16, activation="relu", name="hidden4")
logits = Dense(10, activation="softmax", name="logits")

# 새로운 Transfer Learning 모델 생성
transfer_model = Sequential([
    input_layer,
    hidden1,
    hidden2,
    hidden3,
    hidden4,
    logits
])

# 모델 컴파일
transfer_model.compile(optimizer=Adam(learning_rate=0.001), loss=SparseCategoricalCrossentropy, metrics=['accuracy'])

# MNIST 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# 모델 학습
history = transfer_model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

# 모델 평가 및 테스트 정확도 출력
test_loss, test_acc = transfer_model.evaluate(x_test, y_test)
print(f"테스트 정확도: {test_acc:.4f}")

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6893 - loss: 130.7119 - val_accuracy: 0.9199 - val_loss: 2.3506
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9232 - loss: 1.8374 - val_accuracy: 0.8935 - val_loss: 1.0221
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8920 - loss: 0.9105 - val_accuracy: 0.9266 - val_loss: 0.6167
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9236 - loss: 0.5125 - val_accuracy: 0.9310 - val_loss: 0.4144
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9283 - loss: 0.3715 - val_accuracy: 0.9365 - val_loss: 0.3699
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9196 - loss: 0.9573
테스트 정확도: 0.9310
